In [ ]:
# RAG Implementation - First Steps

This notebook demonstrates the foundational concepts of Retrieval-Augmented Generation (RAG):
1. Setting up embeddings and vector stores
2. Splitting documents into chunks
3. Storing and retrieving similar documents
4. Building a RAG chain with LLM

## Step 1: Install Required Packages

In [ ]:
# Install all required LangChain packages
import subprocess
import sys

packages = [
    "langchain",
    "langchain-chroma",
    "langchain-openai",
    "langchain-core",
    "python-dotenv",
    "chromadb"
]

print("Installing required packages...\n")

for package in packages:
    print(f"Installing {package}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    print(f"✓ {package} installed successfully")

print("\n✓ All packages installed successfully!")
print("\nInstalled packages:")
for package in packages:
    print(f"  - {package}")

## Step 2: Import Required Libraries

In [ ]:
# imports
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter

# load .env file
load_dotenv('../.env')

print("✓ All imports successful")

## Step 3: Initialize Embeddings & ChromaDB Vector Store

ChromaDB is an embedded vector database that stores document embeddings.
We'll use OpenAI's text-embedding-3-large model to create embeddings.